In [ ]:
#Imports
import os
import glob as glob
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

import numpy as np
np.set_printoptions(precision=3, suppress=True)
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns
#Calculate Correlation
from scipy.stats import pearsonr
import os
import glob
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import platform

from calvin_utils.nifti_utils.generate_nifti import view_and_save_nifti

In [ ]:
out_dir = '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/response_topology/voxelwise_mixed_effects/age-stim-interaction_pd-vs-ad'
clinical_data_path = r'/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/clinical_analyses/ses-01/sub-all/ad_pd_mixed_data/ad_pd_full_data.csv'
# clin_path = 'path to clinical values'
print('Will save to: ', out_dir)
save = True
if os.path.exists(out_dir):
    pass
else:
    os.makedirs(out_dir)

# Import Clinical Data

In [ ]:
#----------------------------------------------------------------user input above----------------------------------------------------------------
data_df = pd.read_csv(clinical_data_path)
    
# #Remove subjects
outlier_index=[11, 47, 48, 49]
data_df = data_df.drop(index=outlier_index)
data_df.reset_index(drop=True, inplace=True)

data_df

Get More Clinical Data As Needed

In [ ]:
subject_column = 'Patient # CDR, ADAS'
clinical_information_column = ['Standardized Age', 'One Hot Disease']
outcome_column = 'Standardized Percent Improvement'

#----------------------------------------------------------------DO NOT MODIFY--------------------------------------------------------
clinical_df_1 = pd.DataFrame()
clinical_df_1 = data_df[clinical_information_column]

clinical_df_1['subject_id'] = data_df.loc[:, [subject_column]]
clinical_df_1['outcome'] = data_df.loc[:, [outcome_column]]

# clinical_df_1['subject_id'] = [id.split('_')[0] for id in data_df[subject_column].to_list()] 
clinical_df_1.set_index('subject_id', inplace=True)

# Convert the 'subject_id' column to strings for each DataFrame
clinical_df_1.index = clinical_df_1.index.astype(str)
display(clinical_df_1)

Import Imaging Data

In [ ]:
from calvin_utils.file_utils.import_matrices import import_matrices_from_folder
from calvin_utils.nifti_utils.generate_nifti import nifti_from_matrix
from nimlab import datasets as nimds
import numpy as np
from nilearn import image, plotting, maskers

#get conectivity values of interest
matrix_path = '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/connectivity_data/vta_published_t_connectivity'
matrix = import_matrices_from_folder(matrix_path, file_pattern='*_T*.nii*')

#Standardize matrix via z score
prepped_matrix = pd.DataFrame()
from calvin_utils.statistical_utils.z_score_matrix import z_score_matrix
for column in matrix.columns:
    #Rename while we're at it. Needs to be an integer, as this is the type of patient ID in the clinical data
    print(column)
    new_name = int(column.split('_')[0]) #.split('T')[1]) #<----------------------------------THIS IS A BUG-CREATOR. MUST BE TAILORED TO PATIENT DATA
    #Standardize column by column
    prepped_matrix[new_name] = z_score_matrix(matrix[column])
    #Set values outside brain back to zero
    prepped_matrix[new_name] = np.where(matrix[column] == 0, 0, prepped_matrix[new_name])

#Set patients to those in the clinical data dataframe
prepped_matrix = prepped_matrix.transpose()
prepped_matrix['subject_id'] = [str(col).split('_')[0] for col in prepped_matrix.index]
prepped_matrix.set_index('subject_id', inplace=True)
neuroimaging_df_1 = prepped_matrix
neuroimaging_df_1.index = neuroimaging_df_1.index.astype(str)
#Display results
display(neuroimaging_df_1)

Import More Imaging Data

In [ ]:
#Import Another Dataframe if desired
from calvin_utils.file_utils.import_matrices import import_matrices_from_folder
from calvin_utils.nifti_utils.generate_nifti import nifti_from_matrix
from nimlab import datasets as nimds
import numpy as np
from nilearn import image, plotting, maskers

#get conectivity values of interest
matrix_path = '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_PD_DBS_STN_WURZBURG/vtasCompoundMNI152/subgroup_with_cognitive_scores/cog_decline_patient_connectivity'
neuroimaging_df_2 = import_matrices_from_folder(matrix_path, file_pattern='*T.nii*')
performed_z_score = False 
#----------------------------------------------------------------DO NOT MODIFY!----------------------------------------------------------------
#Standardize matrix via z score
# neuroimaging_df_2 = pd.DataFrame()
# from calvin_utils.z_score_matrix import z_score_matrix
# if performed_z_score:
#     for column in matrix.columns:
#         #Rename while we're at it. Needs to be an integer, as this is the type of patient ID in the clinical data
#         new_name = int(column.split('_')[0])
#         #Standardize column by column
#         prepped_matrix[new_name] = z_score_matrix(matrix[column])
#         #Set values outside brain back to zero
#         prepped_matrix[new_name] = np.where(matrix[column] == 0, 0, prepped_matrix[new_name])

#Set patients to those in the clinical data dataframe
# prepped_matrix = prepped_matrix.loc[:, data_df.index]

neuroimaging_df_2 = neuroimaging_df_2.transpose()
neuroimaging_df_2['subject_id'] = [col.split('_')[0].split('MDST')[1] for col in neuroimaging_df_2.index]
neuroimaging_df_2.set_index('subject_id', inplace=True)

neuroimaging_df_2.index = neuroimaging_df_2.index.astype(str)


#Display results
display(neuroimaging_df_2)

# perform masking

In [ ]:
#Mask Dataframes
mask_path = None #'/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/response_topology/voxelwise_glm/stim_by_age/palm_results/palm_statistic_topology/generated_nifti.nii' #None #
masking_df = None # # 
#----------------------------------------------------------------DO NOT MODIFY
from calvin_utils.nifti_utils.matrix_utilities import mask_matrix
# def mask_matrix(df_1, mask_path=None, mask_threshold=0.2, mask_by='rows', dataframe_to_mask_by=None):

neuroimaging_df_1 = mask_matrix(neuroimaging_df_1, mask_path=mask_path, mask_threshold=0, mask_by='columns', dataframe_to_mask_by=masking_df)
neuroimaging_df_2 = mask_matrix(neuroimaging_df_2, mask_path=mask_path, mask_threshold=0, mask_by='columns', dataframe_to_mask_by=masking_df)


# Perform Standardization

In [ ]:
# Define a lambda function for standardization
from sklearn.preprocessing import StandardScaler

# Standardize the data
scaler = StandardScaler()
neuroimaging_df_1.iloc[:, 0:] = scaler.fit_transform(neuroimaging_df_1.iloc[:, 0:])
neuroimaging_df_2.iloc[:, 0:] = scaler.fit_transform(neuroimaging_df_2.iloc[:, 0:])

In [ ]:
neuroimaging_df_1

## Join Imaging Data to the Clinical Data

In [ ]:
from calvin_utils.file_utils.dataframe_utilities import preprocess_colnames_for_regression
# Stack neuroimaging dataframes
stacked_neuroimaging_df = pd.concat([neuroimaging_df_1, neuroimaging_df_2], axis=0)

# Perform inner join with clinical_df
merged_df = preprocess_colnames_for_regression(clinical_df_1.join(stacked_neuroimaging_df, how='inner'))
merged_df

# Optional - Save Dataframe

In [ ]:
merged_df.to_csv(os.path.join(out_dir, "dataframe_for_mixed_effects.csv"))
merged_df.head(3)

# Run The Mixed Effects Analysis

In [ ]:
from calvin_utils.file_utils.print_suppression import HiddenPrints
import gc
import os
from tqdm import tqdm
import statsmodels.regression.mixed_linear_model as sm
from patsy import dmatrices


def extract_predictors_from_formula(formula):
    y, X = dmatrices(formula, pd.DataFrame({'voxel': [0, 1], 'Standardized_Age': [0, 1], 'outcome': [0, 1]}), return_type='dataframe')
    return X.columns.tolist()[0:]

def initialize_results_array(num_voxels, expected_predictors, metrics):
    num_metrics = len(expected_predictors) * len(metrics) + 1  # +1 for 'voxel'
    return np.zeros((num_voxels, num_metrics))

def voxelwise_mixed_effects_regression_updated(data_df, formula_template, random_effects_column, model_type='linear', batch_size=50000, checkpoint_path='checkpoint.parquet', use_checkpoints=False):
    """
    Perform mixed-effects regression voxelwise based on the provided formula template.
    
    Parameters:
        data_df (pd.DataFrame): DataFrame containing outcome, voxel values, clinical covariates, and other variables with subjects in rows.
        formula_template (str): A string template for the regression formula with 'voxel' as a placeholder for voxel columns.
        voxel_columns (list): List of voxel column names in data_df.
        random_effects_column (str): The column in data_df to be used for random effects.
        model_type (str, default='linear'): Specifies the type of regression model to use ('linear' or 'logistic').
        batch_size (int, default=5000): Number of voxels to process before saving a checkpoint.
        checkpoint_path (str, default='checkpoint.parquet'): Path to save the intermediate results as a checkpoint.
        use_checkpoints (bool, default=False): whether or not to use checkpoint function

    Returns:
        results_df (pd.DataFrame): DataFrame containing p-values, coefficient values, t-values for each voxel,
                                   along with the coefficient, t-value, and p-value for each predictor.
    """
    # Extract predictors and initialize results array
    voxel_columns = data_df.columns[data_df.columns.get_loc('outcome')+1:]
    expected_predictors = extract_predictors_from_formula(formula_template)
    metrics = ['_coeff', '_t_value', '_p_value']
    num_metrics = len(expected_predictors) * len(metrics) + 1  # +1 for 'voxel'
    num_voxels = len(voxel_columns)
    results_array = np.zeros((num_voxels, num_metrics))
    
    # Existing checkpointing logic
    try:
        if (os.path.exists(checkpoint_path)) & (use_checkpoints):
            results_df = pd.read_parquet(checkpoint_path)
            start_idx = len(results_df)
        else:
            start_idx = 0
    except Exception as e:
        print(f"Failed due to error: {e}.")
    
    # Loop through each voxel column and fit the model
    for idx, voxel in enumerate(tqdm(voxel_columns[start_idx:])):
        formula = formula_template.replace('voxel', voxel)
        
        # Existing mixed-effects logic
        try:
            if model_type == 'linear':
                with HiddenPrints():
                    model = sm.MixedLM.from_formula(formula, data=data_df, groups=data_df[random_effects_column]).fit(method="cg")
            else:
                raise ValueError(f"Unsupported model_type: {model_type}")

            # New: Directly populate the results_array
            col_idx = 1
            for predictor in model.params.index:
                results_array[idx, col_idx:col_idx + 3] = [model.params[predictor], model.tvalues[predictor], model.pvalues[predictor]]
                col_idx += 3

        except Exception as e:
            if str(e) == "Singular matrix":
                pass  # Handle singular matrix cases as needed
            
        # Existing checkpointing logic
        try:
            if ((idx + 1) % batch_size == 0) & (use_checkpoints):
                pd.DataFrame(results_array).to_parquet(checkpoint_path)
                gc.collect()
        except Exception as e:
            print(f"Failed to save checkpoint due to error: {e}.")
    
    # Generate DataFrame from results_array
    column_names = ['voxel'] + [f"{pred}{met}" for pred in expected_predictors for met in metrics]
    results_df = pd.DataFrame(results_array, columns=column_names)
    
    return results_df, model

In [ ]:
merged_df.columns

In [ ]:
formula = 'outcome ~ Standardized_Age + voxel  + Standardized_Age:voxel'
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    results_df, model = voxelwise_mixed_effects_regression_updated(merged_df.iloc[:, 0:20], 
                                                formula_template=formula, 
                                                random_effects_column = 'One_Hot_Disease', 
                                                model_type='linear', 
                                                batch_size=100000, 
                                                checkpoint_path='checkpoint.parquet')

View Results

In [ ]:
results_df

# Save the Results to Niftis

In [ ]:
import os
from calvin_utils.nifti_utils.generate_nifti import view_and_save_nifti
from calvin_utils.nifti_utils.matrix_utilities import unmask_matrix

def save_results_as_nifti(results_df, out_dir, mask_path=None, mask_threshold=0.2, unmask_by='rows', dataframe_to_unmask_by=None):
    """
    Save each column in the results DataFrame as a NIFTI file.
    
    Parameters:
        results_df (pd.DataFrame): DataFrame containing various statistical measures for each voxel.
        out_dir (str): Directory where NIFTI files should be saved.
        mask_path (str, optional): Path to the NIFTI mask file to use for unmasking.
        mask_threshold (float, optional): Mask threshold for unmasking.
        unmask_by (str, optional): Direction for unmasking ('rows' or 'columns').
        dataframe_to_unmask_by (pd.DataFrame, optional): DataFrame to use for unmasking.
        
    Returns:
        None
    """
    
    # Ensure the output directory exists
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    
    # Iterate over every column in results_df and generate NIFTI files
    for colname in results_df.columns:
        # Unmask the matrix
        unmasked_df = unmask_matrix(results_df[colname], mask_path=mask_path, mask_threshold=mask_threshold,
                                    unmask_by=unmask_by, dataframe_to_unmask_by=dataframe_to_unmask_by)
        
        # Save the unmasked matrix as a NIFTI file
        view_and_save_nifti(unmasked_df, out_dir, output_name=colname)

In [ ]:
save_results_as_nifti(results_df, out_dir, mask_path=None, mask_threshold=0.2, unmask_by='rows', dataframe_to_unmask_by=None)

Enjoy

-Calvin